# Lunar Lander policy gradient

In [15]:
import gym
import numpy as np
import keras
import gc
import random
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical

from keras.layers import Concatenate, Input, Dense
from keras.layers import merge
from keras.layers import Reshape
from keras.models import Model

import matplotlib.pyplot as plt

MAX_EPISODES = 200
MAX_STEPS = 10000
MAX_BUFFER = 40000
MAX_TOTAL_REWARD = 300

BATCH_SIZE = 100

TAU = 0.0001
LAYER_1 = 128
LAYER_2 = 64
ALPHA = 0.002
GAMMA = 0.99

In [12]:
from collections import deque
class Memory:
    def __init__(self, size):
        self.buffer = deque(maxlen=size)

    def sample(self, size):
        batch = random.sample(self.buffer, size)
        random_index_list = np.random.choice(np.arange(len(self.buffer)), size = size, replace = False)
        sublist =  [self.buffer[index] for index in random_index_list]
        states, actions, rewards, next_states = zip(*sublist)
        return states, actions, rewards, next_states

    def remember(self, experience):
        self.buffer.append(experience)

In [14]:
class Actor():
    def __init__(self, state_dim, action_dim):
        self.model = Sequential()
        self.model.add(Dense(LAYER_1, activation = 'relu', input_shape = (state_dim, ), kernel_initializer='he_uniform'))
        self.model.add(Dense(LAYER_2, activation = 'relu', kernel_initializer='he_uniform'))
        self.model.add(Dense(action_dim, activation = 'linear', kernel_initializer='he_uniform'))
        self.adam = Adam(lr=ALPHA)
        self.model.compile(optimizer=self.adam, loss=self.newLoss)

    def forward(self, state):
        return self.model.predict(np.array(state))

    def update(self, source, degree):
        self.model.set_weights((np.array(source.model.get_weights())*degree)+(np.array(self.model.get_weights())*(1-degree)))

    def train(self, state, errors, batch_size):
        self.model.fit( state, errors, batch_size=batch_size, verbose = 0)

    def newLoss(self, xPred,yPred):
        return np.sum(yPred)

In [5]:
class Critic():
    def __init__(self, state_dim, action_dim):
        self.first_input = Input((state_dim, ))
        self.first_dense = Dense(LAYER_2, activation = 'relu', kernel_initializer='he_uniform')(self.first_input)

        self.second_input = Input((action_dim, ))
        self.second_dense = Dense(LAYER_2, activation = 'relu', kernel_initializer='he_uniform')(self.second_input)

        self.merged = Concatenate()([self.first_dense, self.second_dense])
        self.output_layer = Dense(LAYER_2, activation = 'relu', kernel_initializer='he_uniform')(self.merged)
        self.output_layer_2 = Dense(1, kernel_initializer='he_uniform')(self.output_layer)
        
        self.model = Model(inputs=[self.first_input, self.second_input], outputs=self.output_layer_2)
        self.adam = Adam(lr=ALPHA)
        self.model.compile(optimizer=self.adam, loss='mse')

    def forward(self, state, action):
        return self.model.predict([state, action])

    def update(self, source, degree):
        self.model.set_weights((np.array(source.model.get_weights())*degree)+(np.array(self.model.get_weights())*(1-degree)))

In [6]:
class ActorCritic:
    def __init__(self, state_dim, action_dim, memory, load):
        self.memory = memory
        self.noise = OrnsteinUhlenbeckActionNoise(action_dim)

        self.actor = Actor(state_dim, action_dim)
        self.critic = Critic(state_dim, action_dim)
        self.target_actor = Actor(state_dim, action_dim)
        self.target_critic = Critic(state_dim, action_dim)

        if load != 0:
            self.load_models(load) #load the model

        self.net_update(self.target_actor, self.actor, True)
        self.net_update(self.target_critic, self.critic, True)

    def get_action(self, state, train):
        if train:
            action = self.actor.forward([state])
            noise = np.float32(self.noise.sample())
            return action + noise
        action = self.actor.forward([state])
        return action

    def optimize(self):
        state,action,reward,next_state = self.memory.sample(BATCH_SIZE)

        state = np.array(state)
        action = np.array(action)
        reward = np.array(reward)
        next_state = np.array(next_state)

        next_action = self.target_actor.forward(next_state)

        target = reward[:,np.newaxis] + GAMMA* self.target_critic.forward(next_state, next_action)
        self.critic.model.fit([state,action], target, batch_size=BATCH_SIZE, verbose = 0)

        action = self.actor.forward(state)
        self.actor.train(state, self.critic.forward(state, action), BATCH_SIZE)
        self.net_update(self.target_actor, self.actor, False)
        self.net_update(self.target_critic, self.critic, False)

    def net_update(self,target, source, hard):
        degree = 1
        if not hard: degree = TAU
        target.update(source, degree)

    def save_models(self, episode):
        self.actor.model.save_weights('Models/' + str(episode) + 'actor_weights.h5')
        self.critic.model.save_weights('Models/' + str(episode) + 'critic_weights.h5')

    def load_models(self, episode):
        self.actor.model.load_weights('Models/' + str(episode) + 'actor_weights.h5')
        self.critic.model.load_weights('Models/' + str(episode) + 'critic_weights.h5')
        self.net_update(self.target_actor, self.actor, True)
        self.net_update(self.target_critic, self.critic, True)
        print('Models loaded succesfully')

class OrnsteinUhlenbeckActionNoise:
    def __init__(self, action_dim):
        self.action_dim = action_dim
        self.theta = 0.15
        self.sigma = 0.2
        self.dx = np.zeros(self.action_dim)

    def sample(self):
        self.dx = self.dx + self.theta * (-self.dx) + self.sigma * np.random.randn(len(self.dx))
        return self.dx

In [ ]:
def env_run(env, episode, trainer, memory, train):
    state = env.reset()
    epoch_reward = 0
    print(episode)
    
    for step in range(MAX_STEPS):
        if not train:
            env.render()
        action = trainer.get_action(state, train)[0]
        next_state, reward, done, _ = env.step(action)
        epoch_reward +=reward
        if train:
            if done:
                break
            memory.remember((state, action, reward, next_state))
            state = next_state
            trainer.optimize()
        else:
            if done:
                env.close()
                print("\n Testing agent got a reward of :",epoch_reward)
                break
        state = next_state
    gc.collect()
    if episode%100 == 1:
        trainer.save_models(episode)
    return epoch_reward

def prepopulate_memory(memory, env):
    state = env.reset()
    for _ in range(MAX_BUFFER):
        action = env.action_space.sample()
        next_state, reward, done, _ = env.step(action)
        memory.remember((state, action, reward, next_state))
        if done:
            state = env.reset()
        else:
            state = next_state

def main():

    training = 1
    test_interwal = 5
    load = 0

    env = gym.make('LunarLanderContinuous-v2')
    memory = None

    if training == 1:
        memory = Memory(MAX_BUFFER)
        prepopulate_memory(memory, env)
    rewards = []
    max_reward = 0

    trainer = ActorCritic(env.observation_space.shape[0], env.action_space.shape[0], memory, load)

    for episode in np.arange(MAX_EPISODES):
        if training == 1:
            env_run(env, episode, trainer, memory, True)
        if episode%test_interwal == 0:
            max_reward += env_run(env, episode,trainer, None, False)
            rewards.append(max_reward/((episode/test_interwal)+1))
    plt.plot(rewards)
    plt.show()
    
if __name__ == '__main__':
    main()

0
0

 Testing agent got a reward of : -331.90048389958804
1
2
3
4
5
5

 Testing agent got a reward of : -411.122448715369
6
7
8
9
10
10

 Testing agent got a reward of : -308.77591274040805
11
12
13
14
15
15

 Testing agent got a reward of : -589.9832708198111
16
17
18
19
20
20

 Testing agent got a reward of : -757.7523877404694
21
22
23
24
25
25

 Testing agent got a reward of : -815.5923609687721
26
27
28
29
30
30

 Testing agent got a reward of : -468.9086326959192
31
32
33
34
35
35

 Testing agent got a reward of : -646.9092733093082
36
37
38
39
40
40

 Testing agent got a reward of : -688.4848644763729
41
42
43
44
45
45

 Testing agent got a reward of : -809.2504229733386
46
47
48
49
50
50

 Testing agent got a reward of : -433.4999548161564
51
52
